In [231]:
import importlib
import logging
from IPython.display import display_latex

import astropy.units as u
import astropy.constants as C
from tqdm import tqdm

from datetime import datetime, timedelta

import units.smm as smu
importlib.reload(smu)
# logging config
smu.PumpLogger.setLevel(logging.DEBUG)
smu.SmLogger.setLevel(logging.DEBUG)
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [232]:
canist_ref = smu.GasMixture.pure('N2', 293.15*u.K, 1000*u.L, 90*smu.atm)
canist_ref

GasMixture(T=<Quantity 293.15 K>, V=<Quantity 1000. l>, nus={'N2': <Quantity 3743.41692471 mol>})

In [275]:
class SmModel:
    he_i = None
    teg_pwr = 0*u.J
    def __init__(self):
        self.core_room = smu.GasMixture(smu.T_MINIMAL, 15*smu.CELL_VOLUME, {})
        self.core_room._gr = 15
        self.smcore = smu.Supermatter(self.core_room) #
        # pipe networks with pipelines as base
        self.cooled   = smu.PipeNetwork({'cooled':   smu.GasMixture(0*u.K, 70*u.L)})
        self.cooling  = smu.PipeNetwork({'cooling':  smu.GasMixture(0*u.K, 15680*u.L)})
        self.warm     = smu.PipeNetwork({'warm':     smu.GasMixture(0*u.K, 1890*u.L)})
        self.warm.add('filter1', 1*u.m**3)
        self.warm.add('filter2', 1*u.m**3)
        self.warm.add('nopump', .2*u.m**3) # empty for waste or whatever
        self.scalding = smu.PipeNetwork({'scalding': smu.GasMixture(0*u.K, 315*u.L)})
        # connectors
        self.c_hot =  smu.PipeNetwork({'':canist_ref*1})
        self.c_cold = smu.PipeNetwork({'':canist_ref*1})
        # pumps
        self.po1 = smu.OutletInjector('PO1', P_max=30*u.kW, target_V=700*u.L) # Outlet Injector
        self.pi1 = smu.VentPump('PI1', V_out=700*u.L, ex_P = 100*u.kPa) # , V_in=0*u.L, V_out=0.2*u.m**3) # Vent Pump
        self.hp1 = smu.Pump('HP1', P_max=45*u.kW)
        self.pc_hot =  smu.Pump('P[C]hot') # C2-P2 on pic
        self.pc_cold = smu.Pump('P[C]cold') # C5-P6 on pic
        # TEG and HE
        self.t_hot =  smu.Turbine('T(hot)1')
        self.t_cold = smu.Turbine('T(cold)2')
        self.teg = smu.TEG(self.t_hot, self.t_cold)
        self.he = smu.HeatExchanger(191)
        # -- connecting devices --
        self.pc_hot.connect( self.c_hot,     self.warm)
        self.pc_cold.connect(self.c_cold,    self.cooling)
        self.hp1.connect(    self.cooling,   self.cooled)
        self.pi1.connect(    self.core_room, self.scalding)
        self.po1.connect(    self.warm,      self.core_room)
        self.t_hot.connect(  self.scalding,  self.warm)
        self.t_cold.connect( self.cooled,    self.cooling)

    def energize_core(self, n_pulses: int):
        self.smcore.eer = smu.POWERED_EMITTER_DMG*n_pulses/smu.SM_EER_EMIT

    def pump_cold(self):
        if self.he_i is None: self.he_i = 0
        if self.he_i > 1: 
            self.he.cool(self.cooling.gases['cooling'])
        [x.equalize() for x in [self.c_cold, self.cooling]]
        self.pc_cold.pump()
        pass
        self.he_i += 1

    def pump_hot_teg(self):
        pwr_lost = [x.pump() for x in [self.pc_hot, self.pi1, self.po1]]
        self.teg_pwr = self.teg.process()
        [x.equalize() for x in [self.c_hot, self.warm, self.scalding]]

    def full_tick(self):
        self.pump_cold()
        self.hp1.pump()
        self.pump_hot_teg()
        self.smcore.process()
        [x.equalize() for x in [self.c_hot, self.warm, self.scalding, self.c_cold, self.cooling, self.cooled]]
        self.core_room.react()

    @property
    def reftime(self):
        # t0 = time(0,0,0)
        if self.he_i is None: return timedelta(seconds=0)
        dt = timedelta(seconds=self.he_i*2)
        return (dt)

    def _repr_latex_(self) -> str:
        return '\n\n'.join([
            f'Ticks spent: {self.he_i} ; server time: {self.reftime}; ',
            'Last power: ' + (self.teg_pwr/u.s).to(u.kW).round(2)._repr_latex_(),
            self.smcore._repr_latex_(),
            # 'C[hot]: '  + self.c_hot._repr_latex_()[25:],
            # 'C[cold]: ' + self.c_cold._repr_latex_()[25:],
            'scalding: '+ self.scalding._repr_latex_()[25:],
            'warm: ' +    self.warm._repr_latex_()[25:],
            'cooling: ' + self.cooling._repr_latex_()[25:],
            'cooled: ' +  self.cooled._repr_latex_()[25:],
        ])


In [276]:
m = SmModel()

In [277]:
%config Application.log_level = 10
for _ in range(5):
    m.pump_cold()
%config Application.log_level = 30

[ct.sm.Pump] P[C]cold: Source entropy 155.129 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change 149845 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 0 J / mol
[ct.sm.Pump] P[C]cold: Transferred 623.903 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 0 K, 0 mol, 0 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 293.15 K, 623.9 mol, 7599.37 kPa
[ct.sm.Pump] P[C]cold: Source entropy 156.608 J / (K mol) -> Sink entropy 191.475 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change 34.8669 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 0 J / mol
[ct.sm.Pump] P[C]cold: Transferred 519.919 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 293.15 K, 7.66465 mol, 93.3584 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 293.15 K, 527.6 mol, 6426.17 kPa
[ct.sm] last -9.18464423578679 J; Per 191 pipes 293.15 K -> 293.00 K (0.1461 K loss)
[ct.sm.Pump] P[C]cold: Source entropy 158.094 J / (K mol) -> Sink entropy 186.441 J / (K mol)
[ct.sm.Pump] P[C]cold: Spe

In [236]:
for i in range(3):
    for _ in range(60):
        m.pump_cold()
        m.pump_hot_teg()
    #reload canister twice
    if i < 2:
        m.c_hot.gases['']  = canist_ref*1
        m.c_cold.gases[''] = canist_ref*1
    # print('Cooling [HE]')
    # display_latex(m.cooling.gases['cooling'].p.to(u.kPa).round(2))
    # print('Warm')
    # display_latex(m.warm.gases['warm'].p.to(u.kPa).round(2))
    # print('Core room')
    display_latex(m.smcore)

In [266]:
m.energize_core(8)
m.smcore

Supermatter(room=GasMixture(T=<Quantity 3343.12883709 K>, V=<Quantity 37500. l>, nus={'O2': <Quantity 36.9924316 mol>, 'N2': <Quantity 9382.54697178 mol>, 'Ex*': <Quantity 0. mol>}), eer=<Quantity 113.776 EER>)

In [273]:
T0 = m.smcore.room.T * 1
for i in tqdm(range(60)):
    m.full_tick()

100%|██████████| 60/60 [00:32<00:00,  1.84it/s]


In [274]:
display_latex(m._repr_latex_() + '\n\nRoom :' + m.core_room._repr_latex_(), raw=True)

In [153]:
T0 = m.smcore.room.T * 1
for i in tqdm(range(500)):
    m.full_tick()
    if i % 50 == 0:
        if m.smcore.room.T < T0:
            print('stable-ish?')
            break
        T0 = m.smcore.room.T * 1

100%|██████████| 500/500 [04:10<00:00,  1.99it/s]


In [167]:
print(timedelta(seconds=m.he_i*2))

0:22:40


In [154]:
m.smcore

Supermatter(room=GasMixture(T=<Quantity 2251.35232729 K>, V=<Quantity 37500. l>, nus={'O2': <Quantity 2.77555756e-17 mol>, 'N2': <Quantity 9421.96387231 mol>, 'Ex*': <Quantity 11.00567041 mol>}), eer=<Quantity 111.68782855 EER>)

probably worth it to make a parser of neb debug logs to match with own debug logs

current error for cold loop on its own is sub 1e-6. probably good enough.

together pumps

In [170]:
m.core_room.T

<Quantity 293.15 K>

In [56]:
for _ in range(15):
    ([x.pump() for x in [pc_hot, pc_cold, po1, pi1, hp1]])
    (teg.process())
    smcore.process()
    if i>1 : he.cool(cooling.gases['cooling'])
    [x.equalize() for x in [chot,  warm, scalding, ccold, cooling, cooled]]
    if cT0 is not None:
        print('Core dT', (core_room.T - cT0))
        cT0 = None
    (pc_cold.pump())

print(f'Canisters: {pc_hot.inlet.gases[""].p.to(u.kPa):.3g} hot, {pc_cold.inlet.gases[""].p.to(u.kPa):.3g} cold')
print(f'Core: {smcore.eer:.3f}, {core_room.T:.2f}, {core_room.p.to(u.kPa):.2f}')
print('warm',    warm.gases['warm'].p.to(u.kPa), warm.gases['warm'].T,       '')
print('cooling', cooling.gases['cooling'].p.to(u.kPa), cooling.gases['cooling'].T, '')
print('scalding',scalding.gases['scalding'].p.to(u.kPa), scalding.gases['scalding'].T)
print(cooled.gases['cooled'].p.to(u.kPa))
cT0 = core_room.T

Core dT -0.15667315126302128 K
Canisters: 0.112 kPa hot, 0.104 kPa cold
Core: 99.696 EER, 2650.89 K, 6025.85 kPa
warm 3638.273988585842 kPa 1431.3612617585973 K 
cooling 3025.6115073830306 kPa 559.3343982114376 K 
scalding 5418.442898313371 kPa 2647.7938893836035 K
4393.945905589115 kPa


In [57]:
{k: v/core_room.Nu for k,v in core_room.nus.items()}

{'O2': <Quantity 0.05351593>,
 'Ph': <Quantity 0.02564187>,
 'N2': <Quantity 0.9208422>}

In [73]:
testn = 10000*u.mol
test = smu.GasMixture(2650*u.K, (37.5*u.m**3).to(u.L), {
    'O2': 0.053  * testn,
    'Ex*': 0.0256* testn,
    'N2': 0.921  * testn
})
test.p.to(u.MPa)

<Quantity 5.87005104 MPa>

In [119]:
test.react()
test

GasMixture(T=<Quantity 4668.64044463 K>, V=<Quantity 37500. l>, nus={'O2': <Quantity 146. mol>, 'Ex*': <Quantity 0. mol>, 'N2': <Quantity 9210. mol>})

In [75]:
test.p.to(u.MPa)

<Quantity 6.11907894 MPa>

In [120]:
{k: v/test.Nu for k,v in test.nus.items()}

{'O2': <Quantity 0.01560496>,
 'Ex*': <Quantity 0.>,
 'N2': <Quantity 0.98439504>}